In [2]:
!pip install -q mlxtend
!pip install -q imbalanced-learn
!pip install -q itables
!pip install -q xgboost

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.tree import plot_tree
from mlxtend.plotting import plot_decision_regions
import mlxtend
import sklearn
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and

In [6]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/MyDrive/Colab Notebooks/Machine_learning_Tahlildadeh"
script_path = "/content/drive/MyDrive/Colab Notebooks/Machine_learning_Tahlildadeh"

import sys
sys.path.append(script_path)

from utils import *

df = pd.read_csv(f'{data_path}/diabetes.csv')
print(df.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(768, 9)


In [8]:
df

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                       0.351   31        0  
2                       0.672   32        1  
3                       0.167   21        0  
4                       2.288   33        1  
..                        ...  ...      ...  
763                     0.171   63        0  
764                     0.340   27        0  
765                     0.245   30        0  
766                     0.349   47        1  
767                     0.315   23        0  

[768 rows x 9 columns]

In [9]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
x.shape

(768, 8)

In [10]:
print('Number of missing values:')
for col in df.columns:
    n_MV = sum(df[col].isna())
    print(f'{col}:{n_MV}')

Number of missing values:
Pregnancies:0
Glucose:0
BloodPressure:0
SkinThickness:0
Insulin:0
BMI:0
DiabetesPedigreeFunction:0
Age:0
Outcome:0


In [11]:
# Function to replace outliers using IQR method
def replace_outliers_iqr(df):
    for column in df.columns:
        if df[column].dtype in [np.float64, np.int64]:  # Only process numeric columns
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            upper_cap = Q3 + 1.5 * IQR
            lower_cap = Q1 - 1.5 * IQR

            # Replace values
            df[column] = np.where(df[column] > upper_cap, upper_cap, df[column])
            df[column] = np.where(df[column] < lower_cap, lower_cap, df[column])

    return df

# Separate the 'Outcome' column
outcome = df['Outcome']
cf = df.drop(columns=['Outcome'])

# Apply the function to remove outliers
cleaned_data = replace_outliers_iqr(cf)

# Add the 'Outcome' column back to the cleaned DataFrame
cleaned_data['Outcome'] = outcome

df = cleaned_data

In [12]:
df

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6.0    148.0           72.0           35.0      0.0  33.6   
1            1.0     85.0           66.0           29.0      0.0  26.6   
2            8.0    183.0           64.0            0.0      0.0  23.3   
3            1.0     89.0           66.0           23.0     94.0  28.1   
4            0.0    137.0           40.0           35.0    168.0  43.1   
..           ...      ...            ...            ...      ...   ...   
763         10.0    101.0           76.0           48.0    180.0  32.9   
764          2.0    122.0           70.0           27.0      0.0  36.8   
765          5.0    121.0           72.0           23.0    112.0  26.2   
766          1.0    126.0           60.0            0.0      0.0  30.1   
767          1.0     93.0           70.0           31.0      0.0  30.4   

     DiabetesPedigreeFunction   Age  Outcome  
0                       0.627  50.0        1  
1                       0.351  31.0        0  
2                       0.672  32.0        1  
3                       0.167  21.0        0  
4                       1.200  33.0        1  
..                        ...   ...      ...  
763                     0.171  63.0        0  
764                     0.340  27.0        0  
765                     0.245  30.0        0  
766                     0.349  47.0        1  
767                     0.315  23.0        0  

[768 rows x 9 columns]

In [13]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
x.shape

(768, 8)

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=5,stratify=y)

x_temp_test = x_test.copy()

In [15]:
counter = Counter(y)
print ('Before', counter)

Before Counter({0: 500, 1: 268})


In [16]:
smt = SMOTE(random_state = 5)

In [17]:
x_sm , y_sm = smt.fit_resample(x_train,y_train)

In [18]:
counter = Counter(y_sm)
print('After', counter)

After Counter({1: 400, 0: 400})


In [19]:
from sklearn.metrics import recall_score     #  Feature Selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import itertools


rf = xgboost.XGBClassifier(random_state=1)

columns = list(x_sm.columns)

result = pd.DataFrame(index=range(255), columns=['attrs','score'])
c = 0
for n in range(1, x_sm.shape[1]+1):
    for attrs in itertools.combinations(columns,n):
        xs = x_sm[list(attrs)]
        # gsv = GridSearchCV(rf,param_grid = params,cv = 5, scoring = 'recall')
        rf.fit(xs, y_sm)
        y_pred = rf.predict(x_test[list(attrs)])
        result.iloc[c,0] = attrs
        result.iloc[c,1] = recall_score(y_test, y_pred)
        c += 1

In [20]:
result

attrs     score
0                                       (Pregnancies,)  0.314815
1                                           (Glucose,)  0.555556
2                                     (BloodPressure,)  0.222222
3                                     (SkinThickness,)  0.444444
4                                           (Insulin,)  0.759259
..                                                 ...       ...
250  (Pregnancies, Glucose, BloodPressure, Insulin,...  0.666667
251  (Pregnancies, Glucose, SkinThickness, Insulin,...   0.62963
252  (Pregnancies, BloodPressure, SkinThickness, In...  0.722222
253  (Glucose, BloodPressure, SkinThickness, Insuli...  0.666667
254  (Pregnancies, Glucose, BloodPressure, SkinThic...  0.703704

[255 rows x 2 columns]

In [21]:
selected_columns = ['SkinThickness', 'Insulin', 'BMI', 'Age']
x_sm = x_sm[selected_columns]

x_test = x_test[selected_columns]


In [22]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import numpy as np
from scipy.stats import uniform, randint

xgb_param_grid = {
    'n_estimators': randint(20, 201),
    'learning_rate': uniform(0.01, 0.19),
    'max_depth': randint(3, 8),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

xgb_clf = xgb.XGBClassifier(random_state=1)

random_search_xgb = RandomizedSearchCV(
    xgb_clf,
    param_distributions=xgb_param_grid,
    n_iter=100,
    cv=10,
    verbose=1,
    random_state=1,
    n_jobs=-1
)

random_search_xgb.fit(x_sm, y_sm)

print("Best parameters for XGBoost:", random_search_xgb.best_params_)
print("Best score for XGBoost:", random_search_xgb.best_score_)


Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best parameters for XGBoost: {'colsample_bytree': 0.9147911547905092, 'learning_rate': 0.16252392574809935, 'max_depth': 4, 'n_estimators': 21, 'subsample': 0.9595060755936949}
Best score for XGBoost: 0.735


In [23]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state=42)

param_grid = {
    'subsample': [0.950, 0.960, 0.970],
    'n_estimators': [10, 20, 30],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.15],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=10, scoring='neg_mean_absolute_error', verbose=1)

grid_search.fit(x_sm, y_sm)

best_model = grid_search.best_estimator_

print("Best parameters for XGBoost:", grid_search.best_params_)
print("Best score for XGBoost:", grid_search.best_score_)


Fitting 10 folds for each of 243 candidates, totalling 2430 fits
Best parameters for XGBoost: {'colsample_bytree': 1.0, 'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 30, 'subsample': 0.95}
Best score for XGBoost: -0.3477703344316979


In [24]:
import xgboost as xgb
from pprint import pprint
from utils import clf_score

xgb_clf = xgb.XGBClassifier(
    colsample_bytree =1,
    learning_rate=0.15,
    max_depth=5,
    n_estimators=30,
    subsample=0.95,
    random_state=1
)

xgb_clf.fit(x_sm, y_sm)

y_pred_train = xgb_clf.predict(x_sm)
pprint(clf_score(y_sm, y_pred_train, 'Train'))

y_pred_test = xgb_clf.predict(x_test)
pprint(clf_score(y_test, y_pred_test, 'Test'))




**********************************
Scores for Train data
**********************************
{'accuracy_score': 0.85,
 'f1_score': 0.8594847775175645,
 'precision_score': 0.8083700440528634,
 'recall_score': 0.9175}

**********************************
Scores for Test data
**********************************
{'accuracy_score': 0.6883116883116883,
 'f1_score': 0.6470588235294118,
 'precision_score': 0.5365853658536586,
 'recall_score': 0.8148148148148148}


In [25]:
#x_sm[x_sm == 0] = np.nan     #######################################################################

In [26]:
# Solving Overtrain
import xgboost as xgb
from pprint import pprint

xgb_clf = xgb.XGBClassifier(
    colsample_bytree =1,
    learning_rate=0.15,
    max_depth=3,
    n_estimators=30,
    subsample=1,
    random_state=1
)

xgb_clf.fit(x_sm, y_sm)

y_pred_train = xgb_clf.predict(x_sm)
pprint(clf_score(y_sm, y_pred_train, 'Train'))

y_pred_test = xgb_clf.predict(x_test)
pprint(clf_score(y_test, y_pred_test, 'Test'))




**********************************
Scores for Train data
**********************************
{'accuracy_score': 0.77875,
 'f1_score': 0.7972508591065293,
 'precision_score': 0.7357293868921776,
 'recall_score': 0.87}

**********************************
Scores for Test data
**********************************
{'accuracy_score': 0.7142857142857143,
 'f1_score': 0.6716417910447761,
 'precision_score': 0.5625,
 'recall_score': 0.8333333333333334}


In [27]:
##########################################  Previous codes

In [30]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb

# Define classifiers
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = SVC(kernel='rbf', probability=True, random_state=1)
clf4 = xgb.XGBClassifier(
    colsample_bytree=1,
    learning_rate=0.15,
    max_depth=5,
    n_estimators=30,
    subsample=0.95,
    random_state=1
)

# Combine classifiers in a VotingClassifier
eclf = VotingClassifier(estimators=[
    ('lr', clf1), ('rf', clf2), ('svm', clf3), ('xgb', clf4)], voting='soft')

eclf.fit(x_sm, y_sm)
y_pred_train_eclf = eclf.predict(x_sm)

pprint(clf_score(y_pred_train_eclf, y_sm, 'Train'))

y_pred_test_eclf = eclf.predict(x_test[selected_columns])
pprint(clf_score(y_pred_test_eclf, y_test, 'Test'))



**********************************
Scores for Train data
**********************************
{'accuracy_score': 0.87375,
 'f1_score': 0.8796185935637664,
 'precision_score': 0.9225,
 'recall_score': 0.8405466970387244}

**********************************
Scores for Test data
**********************************
{'accuracy_score': 0.7012987012987013,
 'f1_score': 0.6617647058823529,
 'precision_score': 0.8333333333333334,
 'recall_score': 0.5487804878048781}
